In [1]:
sc = SparkContext.getOrCreate()
sc.version

'2.2.0'

In [2]:
from pyspark.mllib.regression import LabeledPoint
import datetime
start=datetime.datetime.now().time()
print("start time=",start)

def parsePoint(line):
    values = [float(x) for x in line.split(',')]
    return LabeledPoint(values[0], values[1:])

data = sc.textFile(".//irisdata.csv")
XY = data.map(parsePoint)

#print(XY.count())


start time= 12:09:21.675799


In [3]:
ratio=0.85
unlabeledXY, labeledXY = XY.randomSplit([ratio,1-ratio], seed = 100)

#print(unlabeledXY.count())

In [4]:
# Build the model : works on only binary data
from pyspark.mllib.classification import SVMWithSGD,LogisticRegressionWithSGD,LogisticRegressionWithLBFGS

model = LogisticRegressionWithSGD.train(labeledXY)

# Evaluating the model on labeled data
labelAndPred = labeledXY.map(lambda p: (p.label, model.predict(p.features)))
score =1-(labelAndPred.filter(lambda lp: lp[0] != lp[1]).count() / float(labelAndPred.count()+1))
print("Labeled accuracy = ", score)

C:\Spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\mllib\classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


Labeled accuracy =  1.0


In [5]:
# Evaluating the model on unlabeled data
labelAndPred = unlabeledXY.map(lambda p: (p.label, model.predict(p.features)))
score = 1-(labelAndPred.filter(lambda lp: lp[0] != lp[1]).count() / float(labelAndPred.count()+1))

unlabeledProb=unlabeledXY.map(lambda p: sum(model.weights*p.features))
unlabeledProb=unlabeledProb.map(lambda p: 1./(1+np.exp(-p)))
    


In [6]:
import numpy as np

max_iter = 5
probThreshold = 0.9

i = 0
while (i < max_iter and score>0.01):
    
    labelAndPred = unlabeledXY.map(lambda p: (p.label, model.predict(p.features)))
    score = 1-(labelAndPred.filter(lambda lp: lp[0] != lp[1]).count() / float(labelAndPred.count()+1))
    ratio=1-(labeledXY.count()/XY.count())
    print("iteration=",i," Unlabeled ratio=", ratio)

    labelidxy = unlabeledProb.map(lambda p: p >  probThreshold).zipWithIndex().filter(lambda p: p[0]==True).map(lambda p: p[1])
    unlabelidxy = unlabeledProb.map(lambda p: p <=  probThreshold).zipWithIndex().filter(lambda p: p[0]==True).map(lambda p: p[1])

    xx=labelidxy.collect() 
    unlabelAdd=unlabeledXY.zipWithIndex().filter(lambda p: p[1] in xx).map(lambda p: p[0])
    labeledXY=labeledXY.union(unlabelAdd)

    xx=unlabelidxy.collect() 
    unlabeledXY=unlabeledXY.zipWithIndex().filter(lambda p: p[1] in xx).map(lambda p: p[0])
    
    model = LogisticRegressionWithSGD.train(labeledXY)
    unlabeledProb=unlabeledXY.map(lambda p: sum(model.weights*p.features))
    unlabeledProb=unlabeledProb.map(lambda p: 1./(1+np.exp(-p)))
    
    i += 1



iteration= 0  Unlabeled ratio= 0.86


C:\Spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\mllib\classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


iteration= 1  Unlabeled ratio= 0.44999999999999996
iteration= 2  Unlabeled ratio= 0.44999999999999996
iteration= 3  Unlabeled ratio= 0.44999999999999996
iteration= 4  Unlabeled ratio= 0.44999999999999996


In [7]:
end=datetime.datetime.now().time()
print("end time=",end)

def duration(te,ts):
    ds=ts.second+ts.minute*60+ts.hour*3600
    de=te.second+te.minute*60+te.hour*3600
    return(de-ds)
print(duration(end,start))


end time= 12:13:12.251217
231
